In [1]:
import os
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# To get reproducible results
np.random.seed(0)
tf.set_random_seed(0)

print(os.listdir("../input"))

['movie-review-sentiment-analysis-kernels-only', 'fasttext-crawl-300d-2m']


In [2]:
test_file = '../input/movie-review-sentiment-analysis-kernels-only/test.tsv'
test = pd.read_csv(test_file, delimiter='\t').fillna('')
x_test = test.values[:, 2]

train_file = '../input/movie-review-sentiment-analysis-kernels-only/train.tsv'
train = pd.read_csv(train_file, delimiter='\t').fillna('')
x_train = train.values[:, 2]
y_train = train.values[:, 3]

print('x_test count: {}'.format(len(x_test)))
print('x_train count: {}'.format(len(x_train)))
print('y_train count: {}'.format(len(y_train)))

x_test count: 66292
x_train count: 156060
y_train count: 156060


In [3]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_length = 60
max_features = 20000

x_all = []
x_all.extend(x_test)
x_all.extend(x_train)

tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(x_train)
x_test_seq = tk.texts_to_sequences(x_test)

np_x_train = pad_sequences(x_train_seq, maxlen=max_length,  padding='post')
np_x_test = pad_sequences(x_test_seq, maxlen=max_length,  padding='post')
np_y_train = to_categorical(y_train)

print ('np_x_train shape: {}'.format(np_x_train.shape))
print ('np_x_test shape: {}'.format(np_x_test.shape))
print ('np_y_train shape: {}'.format(np_y_train.shape))

Using TensorFlow backend.


np_x_train shape: (156060, 60)
np_x_test shape: (66292, 60)
np_y_train shape: (156060, 5)


In [4]:
import tqdm

word_dict = tk.word_index
embedding_dim = 300
embeddings_index = {}

with open('../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec', 'r') as f:
    lines = f.readlines()

for i in tqdm.tqdm(range(len(lines))):
    values = lines[i].rstrip().rsplit(' ')
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')

max_features = min(max_features, len(word_dict) + 1)
embedding_matrix = np.zeros((max_features, embedding_dim))

for word, i in word_dict.items():
    if i >= max_features:
        break

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print('Embedding matrix: {}'.format(embedding_matrix.shape))

100%|██████████| 2000001/2000001 [02:05<00:00, 15903.73it/s]


Embedding matrix: (19479, 300)


In [5]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping

def one_input_classifier(index, classifier_type, input_length, max_features, class_num, embedding_dim, embedding_matrix):
    inputs = Input(shape=(input_length,), name='input_1')
    embeddings = Embedding(max_features, embedding_dim,
                           weights=[embedding_matrix], input_length=input_length,
                           trainable=False, name='embedding_1')(inputs)
    x = SpatialDropout1D(0.3, name='spatial_dropout1d_1')(embeddings)

    if classifier_type == 'bilstm_cnn':
        x = Bidirectional(CuDNNLSTM(128, name='lstm_1', return_sequences=True), name='bidirectional_1')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Conv1D(128, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(128, 3, activation='relu', name='conv1d_2')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_2')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_3')(x)

    elif classifier_type == 'bilstm_cnn_x2':
        x = Bidirectional(CuDNNLSTM(64, name='lstm_1', return_sequences=True), name='bidirectional_1')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Conv1D(64, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(64, 4, activation='relu', name='conv1d_2')(x)
        x = Conv1D(64, 3, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_2')(x)

        x = Bidirectional(CuDNNLSTM(128, name='lstm_2', return_sequences=True), name='bidirectional_2')(x)
        x = Dropout(0.25, name='dropout_3')(x)
        x = Conv1D(128, 3, activation='relu', name='conv1d_4')(x)
        x = Conv1D(128, 2, activation='relu', name='conv1d_5')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_6')(x)
        x = Dropout(0.25, name='dropout_4')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_5')(x)

    elif classifier_type == 'lstm_cnn':
        x = CuDNNLSTM(128, name='lstm_1', return_sequences=True)(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Conv1D(128, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(128, 3, activation='relu', name='conv1d_2')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_2')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_3')(x)

    elif classifier_type == 'cnn_bilstm':
        x = Conv1D(128, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(128, 3, activation='relu', name='conv1d_2')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Bidirectional(CuDNNLSTM(128, name='lstm_1', return_sequences=True), name='bidirectional_1')(x)
        x = Dropout(0.25, name='dropout_2')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_3')(x)

    elif classifier_type == 'cnn_bilstm_x2':
        x = Conv1D(64, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(64, 4, activation='relu', name='conv1d_2')(x)
        x = Conv1D(64, 3, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Bidirectional(CuDNNLSTM(64, name='lstm_1', return_sequences=True), name='bidirectional_1')(x)
        x = Dropout(0.25, name='dropout_2')(x)

        x = Conv1D(128, 3, activation='relu', name='conv1d_4')(x)
        x = Conv1D(128, 2, activation='relu', name='conv1d_5')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_6')(x)
        x = Dropout(0.25, name='dropout_3')(x)
        x = Bidirectional(CuDNNLSTM(128, name='lstm_2', return_sequences=True), name='bidirectional_2')(x)
        x = Dropout(0.25, name='dropout_4')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_5')(x)

    elif classifier_type == 'cnn_lstm':
        x = Conv1D(128, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(128, 3, activation='relu', name='conv1d_2')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = CuDNNLSTM(128, name='lstm_1', return_sequences=True)(x)
        x = Dropout(0.25, name='dropout_2')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_3')(x)

    elif classifier_type == 'bilstm_only':
        x = Bidirectional(CuDNNLSTM(128, name='lstm_1'), name='bidirectional_1')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_2')(x)

    elif classifier_type == 'lstm_only':
        x = CuDNNLSTM(128, name='lstm_1')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_2')(x)

    elif classifier_type == 'cnn_only':
        x = Conv1D(128, 5, activation='relu', name='conv1d_1')(x)
        x = Conv1D(128, 3, activation='relu', name='conv1d_2')(x)
        x = Conv1D(128, 1, activation='relu', name='conv1d_3')(x)
        x = Dropout(0.25, name='dropout_1')(x)

        x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
        x = Dense(32, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_2')(x)

    elif classifier_type == 'dense_only':
        x = Flatten(name='flatten_1')(x)
        x = Dense(1024, activation='relu', name='dense_1')(x)
        x = Dropout(0.25, name='dropout_1')(x)
        x = Dense(128, activation='relu', name='dense_2')(x)
        x = Dropout(0.25, name='dropout_2')(x)
        x = Dense(32, activation='relu', name='dense_3')(x)
        x = Dropout(0.25, name='dropout_3')(x)

    preds = Dense(class_num, activation='softmax', name='preds')(x)
    model = Model(inputs=inputs, outputs=preds, name='model_{}'.format(index))
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1,
                               mode='min', baseline=None, restore_best_weights=True)

class_num = np_y_train.shape[1]
epochs = 32
batch_size = 1024
validation_split = 0.2
classifier_num = 10

print('Classes: {}'.format(class_num))
print('Epochs: {}'.format(epochs))
print('Batch size: {}'.format(batch_size))
print('Validation split: {:.1}'.format(validation_split))
print('Classifiers: {}'.format(classifier_num))

Classes: 5
Epochs: 32
Batch size: 1024
Validation split: 0.2
Classifiers: 10


In [7]:
classifiers = []
classifier_types = ['bilstm_cnn', 'bilstm_cnn_x2', 'lstm_cnn', 'cnn_bilstm', 'cnn_bilstm_x2', 'cnn_lstm', 'bilstm_only', 'lstm_only', 'cnn_only', 'dense_only']

for i in tqdm.tqdm(range(len(classifier_types))):
    classifiers.append(one_input_classifier(i, classifier_types[i], max_length, max_features,
                                            class_num, embedding_dim, embedding_matrix))

for i in range(classifier_num):
    classifiers[i].summary()
    hist = classifiers[i].fit(np_x_train, np_y_train, validation_split=validation_split,
                              callbacks=[early_stopping], epochs=epochs, batch_size=batch_size, verbose=1)
    classifiers[i].trainable = False

    print('{}'.format(classifier_types[i]))
    print('min loss ({}): {:.4}'.format(i, min(hist.history['loss'])))
    print('min val_loss ({}): {:.4}'.format(i, min(hist.history['val_loss'])))
    print('max acc ({}): {:.4}'.format(i, max(hist.history['acc'])))
    print('max val_acc ({}): {:.4}'.format(i, max(hist.history['val_acc'])))

y_pred_list = []

for i in range(classifier_num):
    y_pred = classifiers[i].predict(np_x_test, batch_size=1024, verbose=1)
    y_pred_list.append(y_pred)

  0%|          | 0/10 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
100%|██████████| 10/10 [00:12<00:00,  1.06s/it]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 60, 300)           5843700   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 60, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 256)           440320    
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 56, 128)           163968    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 54, 128)           49280     
__________

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 124848 samples, validate on 31212 samples
Epoch 1/32
124848/124848 [==============================] - 13s 104us/step - loss: 1.0677 - acc: 0.5717 - val_loss: 0.9488 - val_acc: 0.6106
Epoch 2/32
124848/124848 [==============================] - 9s 73us/step - loss: 0.9267 - acc: 0.6203 - val_loss: 0.9241 - val_acc: 0.6235
Epoch 3/32
124848/124848 [==============================] - 9s 73us/step - loss: 0.8823 - acc: 0.6357 - val_loss: 0.8966 - val_acc: 0.6300
Epoch 4/32
124848/124848 [==============================] - 9s 73us/step - loss: 0.8525 - acc: 0.6472 - val_loss: 0.8990 - val_acc: 0.6331
Epoch 5/32
124848/124848 [==============================] - 9s 73us/step - loss: 0.8247 - acc: 0.6605 - val_loss: 0.8920 - val_acc: 0.6333
Epoch 6/32
124848/124848 [==============================] - 9s 73us/step - loss: 0.8023 - acc: 0.6704 - val_loss: 0.8736 - val_acc: 0.6371
Epoch 7/32
124848/124848 [=========

Train on 124848 samples, validate on 31212 samples
Epoch 1/32
124848/124848 [==============================] - 8s 62us/step - loss: 1.0712 - acc: 0.5744 - val_loss: 0.9593 - val_acc: 0.6088
Epoch 2/32
124848/124848 [==============================] - 5s 44us/step - loss: 0.9293 - acc: 0.6174 - val_loss: 0.9260 - val_acc: 0.6217
Epoch 3/32
124848/124848 [==============================] - 5s 43us/step - loss: 0.8947 - acc: 0.6312 - val_loss: 0.9020 - val_acc: 0.6309
Epoch 4/32
124848/124848 [==============================] - 5s 44us/step - loss: 0.8678 - acc: 0.6432 - val_loss: 0.9431 - val_acc: 0.6252
Epoch 5/32
124848/124848 [==============================] - 5s 44us/step - loss: 0.8433 - acc: 0.6524 - val_loss: 0.8823 - val_acc: 0.6369
Epoch 6/32
124848/124848 [==============================] - 5s 43us/step - loss: 0.8224 - acc: 0.6602 - val_loss: 0.9106 - val_acc: 0.6343
Epoch 7/32
124848/124848 [==============================] - 5s 44us/step - loss: 0.8053 - acc: 0.6671 - val_loss: 0

Epoch 3/32
124848/124848 [==============================] - 9s 76us/step - loss: 0.9341 - acc: 0.6145 - val_loss: 0.9593 - val_acc: 0.6035
Epoch 4/32
124848/124848 [==============================] - 9s 76us/step - loss: 0.8932 - acc: 0.6310 - val_loss: 0.9823 - val_acc: 0.5953
Epoch 5/32
124848/124848 [==============================] - 10s 76us/step - loss: 0.8639 - acc: 0.6435 - val_loss: 0.9296 - val_acc: 0.6069
Epoch 6/32
124848/124848 [==============================] - 9s 76us/step - loss: 0.8402 - acc: 0.6540 - val_loss: 0.9273 - val_acc: 0.6148
Epoch 7/32
124848/124848 [==============================] - 9s 76us/step - loss: 0.8204 - acc: 0.6630 - val_loss: 0.9359 - val_acc: 0.6101
Epoch 8/32
124848/124848 [==============================] - 9s 76us/step - loss: 0.8099 - acc: 0.6663 - val_loss: 0.9195 - val_acc: 0.6134
Epoch 9/32
124848/124848 [==============================] - 9s 76us/step - loss: 0.7905 - acc: 0.6748 - val_loss: 0.9168 - val_acc: 0.6191
Epoch 10/32
124848/124848 

124848/124848 [==============================] - 6s 46us/step - loss: 0.7640 - acc: 0.6825 - val_loss: 0.8748 - val_acc: 0.6406
Restoring model weights from the end of the best epoch
Epoch 00010: early stopping
bilstm_only
min loss (6): 0.764
min val_loss (6): 0.8615
max acc (6): 0.6825
max val_acc (6): 0.6432
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 60, 300)           5843700   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 60, 300)           0         
_________________________________________________________________
lstm_1 (CuDNNLSTM)           (None, 128)               220160    
_________________________________________________________________
dropout_1 (Dropout)         

66292/66292 [==============================] - 2s 24us/step


In [10]:
test_num = np_x_test.shape[0]
y_pred_class = np.ndarray(shape=(test_num,), dtype=np.int32)

for i in range(test_num):
    votes = []

    for j in range(classifier_num):
        vote = y_pred_list[j][i].argmax(axis=0).astype(int)
        votes.append(vote)

    vote_final = max(set(votes), key=votes.count)
    y_pred_class[i] = vote_final

mapping = {phrase: sentiment for _, _, phrase, sentiment in train.values}

# Overlapping
for i, phrase in enumerate(test.Phrase.values):
    if phrase in mapping:
        y_pred_class[i] = mapping[phrase]

test['Sentiment'] = y_pred_class
test[['PhraseId', 'Sentiment']].to_csv('submission.csv', index=False)
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,3
1,156062,8545,An intermittently pleasing but mostly routine ...,3
2,156063,8545,An,2
3,156064,8545,intermittently pleasing but mostly routine effort,3
4,156065,8545,intermittently pleasing but mostly routine,2
